<a href="https://colab.research.google.com/github/MarcSan18/Agentes_ia_alura/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00


In [7]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [16]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [17]:
resp_test= llm.invoke("quem é voce? Seja direto.")
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [18]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [22]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisão: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)


In [24]:
llm_triagem = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [26]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem (mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [28]:
testes = ("posso reembolsar a irtenet?",
          "Quero mais 5 dias de trabalho remoto, Como faço?",
          "Como será o dia amanhã em Ilheus na Bahia?")

In [29]:
for msg_test in testes:
    print(f"Pergunta: {msg_test}\n) -> Resposta:{triagem(msg_test)}|n")

Pergunta: posso reembolssar a irtenet?
) -> Resposta:{'decisão': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}|n
Pergunta: Quero mais 5 dias de trabalho remoto, Como faço?
) -> Resposta:{'decisão': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}|n
Pergunta: Como será o dia amanhã em Ilheus na Bahia?
) -> Resposta:{'decisão': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['contexto_politicas_internas']}|n


# Projeto de Triagem de Service Desk com Langchain e Google Gemini

Este projeto demonstra a criação de um sistema simples de triagem automática para Service Desk utilizando a biblioteca Langchain e o modelo Google Gemini 2.5 Flash. O objetivo é classificar as mensagens dos usuários em categorias como `AUTO_RESOLVER`, `PEDIR_INFO` ou `ABRIR_CHAMADO`, além de determinar a urgência e identificar campos faltantes.

**Nota:** Este projeto ainda está em desenvolvimento e pode sofrer alterações.

## Funcionalidades

- Classificação automática de mensagens de Service Desk.
- Definição de regras de triagem através de um prompt.
- Saída estruturada em formato JSON utilizando Pydantic.
- Integração com o modelo Google Gemini 2.5 Flash.

## Como usar

1.  **Instalar dependências:**